#### 추가 기능 구현
1. path library 사용 - 완료
2. 결과 csv파일 저장 기능 추가 - 완료
3. 유사도 ThreshHold 값 설정 - 수집되는 데이터를 더 보고 결정
4. 프로그램 기능 정리 - 노션 페이지 정리
5. 추가개발 워크스테이션에서 진행 - 컨테이너 생성

In [1]:
import torch
from torchvision import transforms
import cv2
from skimage import io, metrics
from PIL import Image
from pathlib import Path
import re
import csv

/usr/local/lib/python3.8/dist-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.8/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/lib/python3.8/dist-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.8/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
# 워커 이름 설정
worker_name = 'Epoch97_1e4_6490'

In [3]:
# 파일 경로 설정

raw_path = f'/ai-video-converter/outputs'
source_path = f'/ai-video-converter/train/high'
st_csv_path = f'/ai-video-converter/train/check/sim_result_{worker_name}_transformer.csv'
ss_csv_path = f'/ai-video-converter/train/check/sim_result_{worker_name}_ssim.csv'

#### SKlearn을 이용한 SSIM 비교

In [4]:
def image_to_tensor(image_path):
    image = io.imread(image_path)  # 이미지를 읽습니다.
    if image.ndim == 2:  # 흑백 이미지인 경우 채널 차원 추가
        image = image[:, :, None]
    image = transforms.functional.to_tensor(image).float()  # 이미지를 텐서로 변환
    return image.unsqueeze(0)  # 배치 차원 추가

def calculate_ssim(img1, img2, win_size=3):
    if img1.dim() == 3:
        img1 = img1.unsqueeze(0)
    if img2.dim() == 3:
        img2 = img2.unsqueeze(0)

    img1 = img1[0].detach().cpu().numpy().transpose(1, 2, 0)
    img2 = img2[0].detach().cpu().numpy().transpose(1, 2, 0)

    # min_dim = min(img1.shape[0], img1.shape[1])
    # win_size = min(11, min_dim // 2 * 2 + 1)
    
    ssim = metrics.structural_similarity(img1, img2, multichannel=True, full=True, win_size=win_size, data_range=1.0)[0]

    return round(ssim, 4)

In [5]:
def find_files_recursive(root_dir):
    return [file for file in root_dir.rglob('*') if file.is_file()]

def compare_images_ssim(raw_path, source_path, ss_csv_path):
    raw_path = Path(raw_path)
    source_path = Path(source_path)
    ss_csv_path = Path(ss_csv_path)
    

    raw_files = find_files_recursive(raw_path)
    source_files = find_files_recursive(source_path)

    raw_files_dict = {}
    source_files_dict = {}

    for file_path in raw_files:
        match = re.search(r'(\d+)', file_path.name)
        if match:
            raw_files_dict[int(match.group(1))] = file_path

    for file_path in source_files:
        match = re.search(r'(\d+)', file_path.name)
        if match:
            source_files_dict[int(match.group(1))] = file_path

    results = []
    for key in sorted(raw_files_dict.keys()):
        if key in source_files_dict:
            img1 = image_to_tensor(raw_files_dict[key])
            img2 = image_to_tensor(source_files_dict[key])
            ssim_score = calculate_ssim(img1, img2)
            results.append([raw_files_dict[key].name, source_files_dict[key].name, ssim_score])
            print(f"SSIM score between {raw_files_dict[key].name} and {source_files_dict[key].name}: {ssim_score}")


    # 결과를 CSV 파일에 저장
    with ss_csv_path.open('w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Raw Image", "Source Image", "Similarity Score"])
        writer.writerows(results)
        print("Result saved to csv file.")

compare_images_ssim(raw_path, source_path, ss_csv_path)

SSIM score between 2_NOX.jpg and 2.jpg: 0.6177
SSIM score between 5_NOX.jpg and 5.jpg: 0.8423
SSIM score between 6_NOX.jpg and 6.jpg: 0.846
SSIM score between 9_NOX.jpg and 9.jpg: 0.7864
SSIM score between 10_NOX.jpg and 10.jpg: 0.7924
SSIM score between 12_NOX.jpg and 12.jpg: 0.4656
SSIM score between 13_NOX.jpg and 13.jpg: 0.6534
SSIM score between 14_NOX.jpg and 14.jpg: 0.6596
SSIM score between 15_NOX.jpg and 15.jpg: 0.4055
SSIM score between 16_NOX.jpg and 16.jpg: 0.4143
SSIM score between 17_NOX.jpg and 17.jpg: 0.6986
SSIM score between 18_NOX.jpg and 18.jpg: 0.7093
SSIM score between 21_NOX.jpg and 21.jpg: 0.7681
SSIM score between 24_NOX.jpg and 24.jpg: 0.5606
SSIM score between 25_NOX.jpg and 25.jpg: 0.7902
SSIM score between 26_NOX.jpg and 26.jpg: 0.7961
SSIM score between 27_NOX.jpg and 27.jpg: 0.2768
SSIM score between 28_NOX.jpg and 28.jpg: 0.2748
SSIM score between 33_NOX.jpg and 33.jpg: 0.574
SSIM score between 35_NOX.jpg and 35.jpg: 0.6789
SSIM score between 36_NOX.jpg 